## Import


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## fastai import statements for vision not including fastbook

from fastai.vision.all import *
#from fastbook import *

## Import data

In [ ]:
## ../input/herbarium-2020-fgvc7/nybg2020/train/metadata.json
train_path = "../input/herbarium-2020-fgvc7/nybg2020/train/images/"
test_path = "../input/herbarium-2020-fgvc7/nybg2020/test/images/"
image_path = "../input/herbarium-2020-fgvc7/nybg2020/train/images/000/00/437000.jpg"

## From EDA:

Number of Categories, Number of images per category  
[[3, 1],  
 [3726, 2],  
 [2660, 3],  
 [3769, 4],  
 [1434, 5],  
 [1243, 6],  
 [1000, 7],  
 [1833, 8],  
 [757, 9],  
 [683, 10]]  

|  % categories      | n.images /category | Total images       |
|--------------------|--------------------|--------------------|
| 25% (8k)           | 1-4                | 21k                |
| 50% (16k)          | 1-9                | 70k                |
| 75% (24k)          | 1-27               | 198k               |
| 80% (25k)          | 1-36               | 248k               |
| 90% (29k)          | 1-82               | 423k               |
| 100% (32k)         | 1-1795             | 1000k              |

In [ ]:
## Import dfs
df_all = pd.read_csv("../input/processed-2/df_all.csv")
df_all.drop("Unnamed: 0",axis=1, inplace=True)
df_all_cat = pd.read_csv("../input/processed-2/df_one_cat.csv")
df_all.shape,df_all_cat.shape


In [ ]:
## Determin size of dataset
## Reduce size of dater (takes a long time 60s)
df_red = df_all.groupby("category_id").head(n=8)
#df_red = df_all.groupby("category_id").apply(lambda x: x.sample(min(20,len(x)))).reset_index(drop=True)
print("Number of categories:",len(df_red["category_id"].unique()), "\nLength of df:", len(df_red))
df_red
df_red[df_red["category_id"]==23718]

## Managing distribution of data for training

In [ ]:
## declarations
import random
random.seed(42)

use_all_cat = True
min_specimens = 1
max_specimens = None


In [ ]:
## Keep max 8 images per category and keep all cats
df = df_all.groupby("category_id").apply(lambda x: x.sample(min(8,len(x)))).reset_index(drop=True)
df.sort_values("len_rows", inplace=True)
df.reset_index(drop=True, inplace=True)
#df = df[(df["len_rows"] >= min_specimens) & (df["len_rows"] < max_specimens)]
df.reset_index(drop=True, inplace=True)
ctg_unq = list(df["category_id"].unique())

In [ ]:
## Determine valid and test dataset

train_ind_1 = [random.sample(df[df["category_id"]==ctg].index.tolist(),1)[0] for ctg in ctg_unq]

## get rest of train and valid dset
avl_ind = list(set(range(len(df)))-set(train_ind_1))
valid_ind = random.sample(avl_ind,int(0.2*len(df)))
train_ind = list(set(range(len(df)))-set(valid_ind))
print("Valid ind is not in Train indices?", not set(valid_ind).issubset(set(train_ind)))
print("Total images:", len(df_all), "\nTotal filtered images:", len(df), 
      "\nTotal selected cat:", len(train_ind_1), "\nSingle images in df:", len(df[df["len_rows"]==1]))
print("Validation count:", len(valid_ind), "\nTraining count:", len(train_ind))

In [ ]:
## check: All of valid_cat should be in train_cat
valid_set = set(df.loc[valid_ind,"category_id"])
train_set = set(df.loc[train_ind,"category_id"])
print("All Valid categ in Train categories?", valid_set.issubset(train_set))
print("\nTotal selected cat:", len(train_ind_1), 
      "\nTotal select valid categories: ", len(valid_set),"\nTotal select Train categories:",len(train_set))
print(len(valid_set.intersection(train_set)))

In [ ]:
## Make a new column "Is_valid"
df.loc[valid_ind,["is_valid"]] = True
df.loc[train_ind,["is_valid"]] = False
df.sample(10)

In [ ]:
def print_cpu_gpu_usage():
    !gpustat -cp
    !free -m
    #!top -bn1 | grep "Cpu(s)" | sed "s/.*, *\([0-9.]*\)%* id.*/\1/" | awk '{print 100 - $1"%"}'

## Data Block

In [ ]:
## Writting the splitter so that valid data has categories as in train

def splitter(df):
    train_ind = df.index[df['is_valid']==False].tolist()
    valid_ind = df.index[df['is_valid']==True].tolist()
    
    valid_cats = set([df["category_id"].iloc[i] for i in valid_ind])
    train_cats = set([df["category_id"].iloc[i] for i in train_ind])
    if not valid_cats.issubset(train_cats):
        raise Exception("something is wrong")
    return train_ind,valid_ind

train,valid = splitter(df)
len(train), len(valid)

In [ ]:
def get_x(r): return "../input/herbarium-2020-fgvc7/nybg2020/train/"+r["filepath"]
def get_y(r): return r["category_id"]
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),#documentation???
    get_x = get_x,
    get_y = get_y,
    splitter=splitter,
    item_tfms=Resize(256))
    #item_tfms=RandomResizedCrop(256, min_scale=0.08),
    #batch_tfms=aug_transforms(size=224, min_scale=0.5, mult=2, pad_mode='zeros')) # next iter mult=2

    


In [ ]:
## Create dsets and dls
dsets = dblock.datasets(df)
dls = dblock.dataloaders(df,bs=128)
#x,y = dsets.train[0]
#x,y,x.shape,y.shape, len(dsets.train)
#dblock.summary(df)

In [ ]:
## Showing one batch prep
x1,y1 = dls.train.one_batch()
dls.show_batch(nrows=2,ncols=3)
#x.shape,y.shape

In [ ]:
## NN Learner
from fastai.callback.fp16 import *

f1_score_multi = F1Score(average="macro") ## convert class to functie
learn = cnn_learner(dls,resnet50,metrics=f1_score_multi).to_fp16() 

In [ ]:
## check memory usage
print_cpu_gpu_usage()
learn.fine_tune(9, base_lr=3e-3, freeze_epochs=1)
print_cpu_gpu_usage()

In [ ]:
learn.lr

In [ ]:
## Export
learn.export()

In [ ]:
# Size  of files and folders
!ls -l export.pkl
!ls -l df.csv
!du -sh 